In [19]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [4]:
# Load in datasets
accessions = pd.read_excel('data/all-accn.xlsx')
localities = pd.read_excel('data/all-mesozoic.xlsx')
specimens = pd.read_excel('data/spec-meso.xlsx')

c:\ProgramData\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\ProgramData\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\ProgramData\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


I'm going to clean the Specimens data table first! 

Some of these columns I drop because the variables are presented on other datatables, in a way that makes much more sense from their relationship. For example, the "Formation" columnn will be its own table, which links with the Locality table, rather than the specimen table. Other columns contain variables that aren't useful for this project. 

In [5]:
# Drop columns
specimens_clean = specimens.drop(columns=['Last Name', 'Storage/Full Name', 'Erathem/Era', 'Formation', 'System/Period', 'Series/Epoch', 'Name'])

In [6]:
specimens_clean.head()

,SpecNo,AccNo,LocNo,Field No,Taxon/Full Name,Element,Portion,Description
0,21047,12.0,V--1994093,1993 DLL 8-9-2,Vertebrata,NaN,NaN,misc. bones and shell
1,21048,12.0,V--1994093,1993 DLL 8-9-2,Pinales,NaN,NaN,cones of cf. Metasequoia
2,6511,218.0,-I-1999007,NaN,Animalia,NaN,NaN,Growth of ferruginous oncoliths.
3,12773,330.0,V--2008017,2008-23,Ornithischia,NaN,NaN,NaN
4,12774,330.0,V--2008017,2008-23,Hadrosauridae,NaN,NaN,NaN


In [7]:
localities_clean = localities.drop(columns=['Elev (ft)', 'Series/Epoch', 'Stage/Age', 'System/Period', 'Datum', 'Township', 'Range', 'Section', 'Subdiv', 'UTM Band', 'UTM Easting', 'UTM Northing', 'Erathem/Era', 'Map', 'UTM Zone'])

In [ ]:
# Round Latitude1 and Longitude1 to two decimal places, since im not supposed to reveal the coordinates
localities_clean['Latitude1'] = localities_clean['Latitude1'].round(2)
localities_clean['Longitude1'] = localities_clean['Longitude1'].round(2)

# Display the updated DataFrame
localities_clean.head()

,LocNo,Name,State,County,Latitude1,Longitude1,Formation,Locality Description
0,-I-2001012,Mesozoic Invertebrate Unknowns,NaN,NaN,NaN,NaN,NaN,"""This locality is designated for Mesozoic spec..."
1,V--2009007,England Mesozoic General,NaN,NaN,NaN,NaN,NaN,General locality created to accommodate verteb...
2,--P2001009,Mesozoic Plant Unknowns,NaN,NaN,NaN,NaN,NaN,"""This locality is for all Mesozoic plant speci..."
3,V--2013049,Clark Metatarsal,Utah,Garfield County,37.55,-111.81,Kaiparowits,NaN
4,V--2000007,Cretaceous Hell Creek Fm,Montana,NaN,NaN,NaN,Hell Creek,"""Locality includes all specimens which have no..."


In [9]:
accessions.head()

,AccNo,Date Received,Purp of Coll,Material,Date Received (Year)
0,340,02/26/2009,cast specimen for study; purchased on NSF gran...,cast of Diabloceratops skull,2009.0
1,139,02/25/1997,General accession of Mississippian fossils fro...,"Invertebrate fossils from Lakeview, Montana",1997.0
2,140,02/28/1997,General accession of Upper Devonian fossils fr...,"Invertebrate fossils from Rockford, Iowa",1997.0
3,141,02/28/1997,General accession of Devonian fossils from Syl...,"Invertebrate fossils from Sylvania, Ohio",1997.0
4,142,03/04/1997,General accession of Late Cretaceous fossils f...,"Invertebrate fossils from Pierre Shale, South ...",1997.0


In [10]:
accessions_clean = accessions.dropna(subset=['Date Received (Year)'])

In [12]:
# Extract the year as an integer
accessions_clean.loc[:, 'Date Received (Year)'] = accessions_clean['Date Received (Year)'].astype(int)
print(accessions_clean['Date Received (Year)'].head())

0    2009
1    1997
2    1997
3    1997
4    1997
Name: Date Received (Year), dtype: int32


In [13]:
localities.columns

Index(['LocNo', 'Name', 'State', 'County', 'Latitude1', 'Longitude1',
       'Township', 'Range', 'Section', 'Subdiv', 'Elev (ft)', 'Datum',
       'UTM Zone', 'UTM Band', 'UTM Easting', 'UTM Northing', 'Map',
       'Formation', 'Series/Epoch', 'Stage/Age', 'System/Period',
       'Erathem/Era', 'Locality Description'],
      dtype='object')

In [14]:
formations = localities[['Formation', 'Series/Epoch', 'System/Period', 'Erathem/Era', 'Stage/Age']]

In [15]:
formations_clean = formations.dropna(subset=['Formation'])
print(formations_clean.head())

     Formation Series/Epoch System/Period Erathem/Era Stage/Age
3  Kaiparowits          NaN    Cretaceous    Mesozoic       NaN
4   Hell Creek          NaN    Cretaceous    Mesozoic       NaN
5  Kaiparowits          NaN    Cretaceous    Mesozoic       NaN
6  Kaiparowits          NaN    Cretaceous    Mesozoic       NaN
7  Kaiparowits          NaN    Cretaceous    Mesozoic       NaN


In [18]:
# Function to convert column names to lower snake case
def to_snake_case(df):
    df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('/', '_')
    return df

# Apply the function to all of the DataFrames
localities_clean = to_snake_case(localities_clean)
specimens_clean = to_snake_case(specimens_clean)
accessions_clean = to_snake_case(accessions_clean)
formations_clean = to_snake_case(formations_clean)

# Display the updated column names for verification
print(localities_clean.columns)
print(specimens_clean.columns)
print(accessions_clean.columns)
print(formations_clean.columns)

Index(['locno', 'name', 'state', 'county', 'latitude1', 'longitude1',
       'formation', 'locality_description'],
      dtype='object')
Index(['specno', 'accno', 'locno', 'field_no', 'taxon_full_name', 'element',
       'portion', 'description'],
      dtype='object')
Index(['accno', 'date_received', 'purp_of_coll', 'material',
       'date_received_(year)'],
      dtype='object')
Index(['formation', 'series_epoch', 'system_period', 'erathem_era',
       'stage_age'],
      dtype='object')


I've cleaned the data and ended up with my four tables to insert into my database: 
1. specimens_clean
2. localities_clean
3. accessions_clean
4. formations_clean

In the next code chunk I will save them as csv's, to be input into my database

In [22]:
# Define the outputs folder path
outputs = os.path.join(os.getcwd(), 'outputs')

# Save CSVs with a placeholder for NaN values
localities_clean.to_csv(os.path.join(outputs, 'localities_clean.csv'), index=False, na_rep='NULL')
specimens_clean.to_csv(os.path.join(outputs, 'specimens_clean.csv'), index=False, na_rep='NULL')
accessions_clean.to_csv(os.path.join(outputs, 'accessions_clean.csv'), index=False, na_rep='NULL')
formations_clean.to_csv(os.path.join(outputs, 'formations_clean.csv'), index=False, na_rep='NULL')